In [1]:
import numpy as np # linear algebra
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl

import operator 
import random

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mean_squared_error as mse_loss

from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

/home/febriyanto/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/febriyanto/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/febriyanto/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/febriyanto/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym o

In [2]:
data=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

# Modeling

In [3]:
data.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,24,24,40,40,74,84,94,110,110,120
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,70,76,89,104,123,146,174,186,197,212
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,90,139,201,230,264,302,367,409,454,511


In [4]:
confirmed_df=data
cols = confirmed_df.keys()
confirmed=data.loc[:, cols[4]:cols[-1]]
dates = confirmed.keys()

In [5]:
confirmed_df.head(),20,22

(  Province/State Country/Region      Lat     Long  1/22/20  1/23/20  1/24/20  \
 0            NaN    Afghanistan  33.0000  65.0000        0        0        0   
 1            NaN        Albania  41.1533  20.1683        0        0        0   
 2            NaN        Algeria  28.0339   1.6596        0        0        0   
 3            NaN        Andorra  42.5063   1.5218        0        0        0   
 4            NaN         Angola -11.2027  17.8739        0        0        0   
 
    1/25/20  1/26/20  1/27/20  ...  3/20/20  3/21/20  3/22/20  3/23/20  \
 0        0        0        0  ...       24       24       40       40   
 1        0        0        0  ...       70       76       89      104   
 2        0        0        0  ...       90      139      201      230   
 3        0        0        0  ...       75       88      113      133   
 4        0        0        0  ...        1        2        2        3   
 
    3/24/20  3/25/20  3/26/20  3/27/20  3/28/20  3/29/20  
 0     

In [6]:
confirmed_df['first_1']=confirmed_df['1/22/20']
confirmed_df['first_2']=confirmed_df['2/13/20']
confirmed_df['first_3']=confirmed_df['2/15/20']
confirmed_df['mid_1']=confirmed_df['3/13/20']
confirmed_df['mid_2']=confirmed_df['3/15/20']
confirmed_df['mid_3']=confirmed_df['3/17/20']
confirmed_df['last_1']=confirmed_df['3/29/20']
confirmed_df['last_2']=confirmed_df['3/27/20']
confirmed_df['last_3']=confirmed_df['3/24/20']

In [7]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/29/20,first_1,first_2,first_3,mid_1,mid_2,mid_3,last_1,last_2,last_3
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,120,0,0,0,7,16,22,120,110,74
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,212,0,0,0,33,42,55,212,186,123
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,511,0,0,0,26,48,60,511,409,264
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,334,0,0,0,1,1,39,334,267,164
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,7,0,0,0,0,0,0,7,4,3


In [8]:
full_table = confirmed_df.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long","first_1","first_2",
                                        "first_3","mid_1","mid_2","mid_3","last_1","last_2","last_3"], var_name="Date", value_name="Confirmed")

full_table['Date'] = pd.to_datetime(full_table['Date'])
full_table.tail()

,Province/State,Country/Region,Lat,Long,first_1,first_2,first_3,mid_1,mid_2,mid_3,last_1,last_2,last_3,Date,Confirmed
17199,NaN,Burma,21.9162,95.9560,0,0,0,0,0,0,10,8,0,2020-03-29,10
17200,Anguilla,United Kingdom,18.2206,-63.0686,0,0,0,0,0,0,2,0,0,2020-03-29,2
17201,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,2,0,0,2020-03-29,2
17202,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,0,0,0,0,0,0,4,0,0,2020-03-29,4
17203,NaN,MS Zaandam,0.0000,0.0000,0,0,0,0,0,0,2,0,0,2020-03-29,2


In [9]:
len(full_table)

17204

In [10]:
data=full_table.drop(['Lat','Long'],axis=1)

In [11]:
data['days']=(data['Date']-pd.to_datetime("2020-01-22")).dt.days

In [12]:
data.sample(4)

,Province/State,Country/Region,first_1,first_2,first_3,mid_1,mid_2,mid_3,last_1,last_2,last_3,Date,Confirmed,days
278,NaN,Bhutan,0,0,0,1,1,1,4,3,2,2020-01-23,0,1
7602,South Australia,Australia,0,2,2,16,20,29,299,257,170,2020-02-21,2,30
7377,Newfoundland and Labrador,Canada,0,0,0,0,1,3,135,102,35,2020-02-20,0,29
15510,Tianjin,China,4,119,122,136,136,136,166,155,145,2020-03-23,141,61


In [13]:
data['Confirmed'].max()

140886

In [14]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()

categoricals=["Province/State", 	"Country/Region"]
numericals=["days","first_1","first_2","first_3","mid_1","mid_2","mid_3","last_1","last_2","last_3"]

In [15]:
data.fillna(value="no",inplace=True)
data.tail()

,Province/State,Country/Region,first_1,first_2,first_3,mid_1,mid_2,mid_3,last_1,last_2,last_3,Date,Confirmed,days
17199,no,Burma,0,0,0,0,0,0,10,8,0,2020-03-29,10,67
17200,Anguilla,United Kingdom,0,0,0,0,0,0,2,0,0,2020-03-29,2,67
17201,British Virgin Islands,United Kingdom,0,0,0,0,0,0,2,0,0,2020-03-29,2,67
17202,Turks and Caicos Islands,United Kingdom,0,0,0,0,0,0,4,0,0,2020-03-29,4,67
17203,no,MS Zaandam,0,0,0,0,0,0,2,0,0,2020-03-29,2,67


In [16]:
data["Province/State"]=le1.fit_transform(data["Province/State"])
data["Country/Region"]=le2.fit_transform(data["Country/Region"])

In [17]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()
scaler4 = StandardScaler()
scaler5 = StandardScaler()
scaler6 = StandardScaler()
scaler7 = StandardScaler()
scaler8 = StandardScaler()
scaler9 = StandardScaler()
scaler10 = StandardScaler()
scaler11 = StandardScaler()

In [18]:
data[["Confirmed"]]=scaler1.fit_transform(data[["Confirmed"]].to_numpy())
data[["days"]]=scaler2.fit_transform(data[["days"]].to_numpy())
data[["first_1"]]=scaler3.fit_transform(data[["first_1"]].to_numpy())
data[["first_2"]]=scaler4.fit_transform(data[["first_2"]].to_numpy())
data[["first_3"]]=scaler5.fit_transform(data[["first_3"]].to_numpy())

data[["mid_1"]]=scaler6.fit_transform(data[["mid_1"]].to_numpy())
data[["mid_2"]]=scaler7.fit_transform(data[["mid_2"]].to_numpy())
data[["mid_3"]]=scaler8.fit_transform(data[["mid_3"]].to_numpy())

data[["last_1"]]=scaler9.fit_transform(data[["last_1"]].to_numpy())
data[["last_2"]]=scaler10.fit_transform(data[["last_2"]].to_numpy())
data[["last_3"]]=scaler11.fit_transform(data[["last_3"]].to_numpy())

In [19]:
data=data.drop("Date",axis=1)
data.head()

,Province/State,Country/Region,first_1,first_2,first_3,mid_1,mid_2,mid_3,last_1,last_2,last_3,Confirmed,days
0,79,0,-0.078586,-0.078829,-0.077271,-0.126362,-0.138392,-0.153850,-0.200891,-0.202306,-0.199976,-0.10881,-1.706764
1,79,1,-0.078586,-0.078829,-0.077271,-0.120567,-0.132821,-0.147144,-0.194112,-0.195426,-0.193768,-0.10881,-1.706764
2,79,2,-0.078586,-0.078829,-0.077271,-0.122127,-0.131535,-0.146128,-0.172080,-0.175241,-0.175903,-0.10881,-1.706764
3,79,3,-0.078586,-0.078829,-0.077271,-0.127699,-0.141606,-0.150396,-0.185122,-0.188095,-0.188573,-0.10881,-1.706764
4,79,4,-0.078586,-0.078829,-0.077271,-0.127922,-0.141820,-0.158321,-0.209218,-0.211900,-0.208972,-0.10881,-1.706764


In [20]:
val=data[data.days>61]
train=data[data.days<62]

In [21]:
for col in train.columns:
  #print(col)
  print("the number of unique values in "+col +" is "+str(len(data[col].value_counts())))

the number of unique values in Province/State is 80
the number of unique values in Country/Region is 177
the number of unique values in first_1 is 11
the number of unique values in first_2 is 48
the number of unique values in first_3 is 47
the number of unique values in mid_1 is 107
the number of unique values in mid_2 is 120
the number of unique values in mid_3 is 133
the number of unique values in last_1 is 195
the number of unique values in last_2 is 189
the number of unique values in last_3 is 167
the number of unique values in Confirmed is 1267
the number of unique values in days is 68


In [22]:
def model(dense_dim_1=16, dense_dim_2=16, dense_dim_3=16, dense_dim_4=8, 
dropout1=0.4, dropout2=0.3, dropout3=0.3, dropout4=0.4, lr=0.0005,pre_model=None):

    #Inputs   #16,16,16,8
    state = Input(shape=[1], name="Province/State")
    country = Input(shape=[1], name="Country/Region")
    days = Input(shape=[1], name="days")
    
    first_1=Input(shape=[1], name="first_1")
    first_2 =Input(shape=[1], name="first_2")
    first_3 =Input(shape=[1], name="first_3")

    mid_1 =Input(shape=[1], name="mid_1")
    mid_2 =Input(shape=[1], name="mid_2")
    mid_3 =Input(shape=[1], name="mid_3")

    last_1 =Input(shape=[1], name="last_1")
    last_2 	=Input(shape=[1], name="last_2")
    last_3 	=Input(shape=[1], name="last_3")
   
    #Embeddings layers
    emb_state = Embedding(80, 4)(state)
    emb_country = Embedding(177, 8)(country) #change these dimension based on number of unique countries and states
    

    concat_emb = concatenate([
            Flatten() (emb_state), 
            Flatten() (emb_country),
         
    ])
    
    categ = Dropout(dropout1)(Dense(dense_dim_1,activation='relu') (concat_emb))
    categ = BatchNormalization()(categ)
    categ = Dropout(dropout2)(Dense(dense_dim_2,activation='relu') (categ))
    
    #main layer
    main_l = concatenate([
          categ
        , days,first_1 ,first_2, first_3,	mid_1, mid_2,	mid_3, last_1, last_2,
         	last_3 	        
    ])
    
    main_l = Dropout(dropout3)(Dense(dense_dim_3,activation='relu') (main_l))
    main_l = BatchNormalization()(main_l)
    main_l = Dropout(dropout4)(Dense(dense_dim_4,activation='relu') (main_l))
    
    #output
    output = Dense(1) (main_l)

    model = Model([ state,
                    country, 
                    days,first_1 ,first_2, first_3,	mid_1, mid_2,	mid_3, last_1, last_2,last_3 ], output)

    model.compile(optimizer = Adam(lr=lr),
                  loss= mse_loss,
                  metrics=[root_mean_squared_error])
    return model

In [23]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [24]:
def get_keras_data(df, num_cols, cat_cols):
    cols = num_cols + cat_cols
    X = {col: np.array(df[col]) for col in cols}
    return X

In [25]:
Y_train=train.Confirmed
X_train=train.drop('Confirmed',axis=1)
Y_valid=val.Confirmed
X_valid=val.drop("Confirmed",axis=1)

In [26]:
X_t = get_keras_data(X_train, numericals, categoricals)
X_v = get_keras_data(X_valid, numericals, categoricals)

In [27]:
keras_model=model(lr=0.005)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
early_stopping = EarlyStopping(patience=107, verbose=2)
model_checkpoint = ModelCheckpoint("corona.hdf5",
                                       save_best_only=True, verbose=2, monitor='val_root_mean_squared_error', mode='min')
hist = keras_model.fit(X_t, Y_train, batch_size=7000, epochs=300, #bs=32,
                            validation_data=None, validation_split=0.15,
                            callbacks=[early_stopping, model_checkpoint])

Train on 14623 samples, validate on 2581 samples
Epoch 1/300
 7000/14623 [=============>................] - ETA: 0s - loss: 3.6730 - root_mean_squared_error: 1.9165
Epoch 00001: val_root_mean_squared_error improved from inf to 2.16796, saving model to corona.hdf5
14623/14623 [==============================] - 1s 88us/sample - loss: 3.0193 - root_mean_squared_error: 1.5929 - val_loss: 4.7000 - val_root_mean_squared_error: 2.1680
Epoch 2/300
 7000/14623 [=============>................] - ETA: 0s - loss: 2.2844 - root_mean_squared_error: 1.5114
Epoch 00002: val_root_mean_squared_error improved from 2.16796 to 1.91915, saving model to corona.hdf5
14623/14623 [==============================] - 0s 12us/sample - loss: 2.0608 - root_mean_squared_error: 1.3932 - val_loss: 3.6831 - val_root_mean_squared_error: 1.9191
Epoch 3/300
 7000/14623 [=============>................] - ETA: 0s - loss: 1.4790 - root_mean_squared_error: 1.2161
Epoch 00003: val_root_mean_squared_error improved from 1.91915 to

In [29]:
Modl=load_model("corona.hdf5",custom_objects={'root_mean_squared_error': root_mean_squared_error})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [30]:
def predict(Country,State=None,day=70):
  entry=pd.DataFrame()
  if not State:
    entry['Province/State']=["no"]
  entry['Country/Region']=[Country]
  entry['days']=[day]

  
  try:
    entry['Province/State']=le1.transform(entry["Province/State"])
    entry["Country/Region"]=le2.transform(entry["Country/Region"])
    for col in numericals:
      entry[col]=data[(data['Province/State']==entry['Province/State'].loc[0]) & (data['Country/Region']==entry["Country/Region"].loc[0])].iloc[0][col]
    entry[["days"]]=scaler2.transform(entry[["days"]].to_numpy())
    for_prediction=get_keras_data(entry,numericals,categoricals)
    result=Modl.predict(for_prediction)
    result=scaler1.inverse_transform(result)
    print("Number of cases will be "+ str(int(result)))
 

  except:
    print("Enter the Country and State which are in dataset")

In [31]:
predict("Indonesia")

Number of cases will be 1001


In [32]:
predict("Italy")

Number of cases will be 7148


In [33]:
predict("Albania")

Number of cases will be 1081
